In [8]:
import networkx as nx
import numpy as np
import pandas as pd

In [9]:
from node2vec import Node2Vec

In [10]:
from tqdm.notebook import tqdm

In [11]:
from collections import defaultdict

# Big test

In [12]:
dff = pd.read_csv("/home/rafael/googledrive/DOC/data/ccf_preprocessed.csv", index_col=0)

In [13]:
df = dff.iloc[:1000]
G_large = nx.Graph()
G_large.add_nodes_from(df.TX_ID)
G_large.add_nodes_from(df.CARD_PAN_ID.unique())
G_large.add_nodes_from(df.TERM_MIDUID.unique())
G_large.add_edges_from(zip(df.TX_ID, df.TERM_MIDUID))
G_large.add_edges_from(zip(df.TX_ID, df.CARD_PAN_ID))

In [14]:
G_large = nx.convert_node_labels_to_integers(G_large, label_attribute='node_id')

In [17]:
# How long does it take with Eliorc node2vec?
n2v = Node2Vec(G_large, workers=8)

Computing transition probabilities: 100%|██████████| 2199/2199 [00:00<00:00, 18754.57it/s]


In [18]:
n2v.d_graph

defaultdict(dict,
            {0: {'probabilities': {1000: array([0.5, 0.5]),
               1871: array([0.5, 0.5])},
              'first_travel_key': array([0.5, 0.5]),
              'neighbors': [1871, 1000]},
             1871: {'probabilities': {0: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.11111111, 0.11111111, 0.11111111]),
               225: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.11111111, 0.11111111, 0.11111111]),
               272: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.11111111, 0.11111111, 0.11111111]),
               339: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.11111111, 0.11111111, 0.11111111]),
               481: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.111111

In [19]:
A = nx.adjacency_matrix(G_large)
d_graph = dict()
FIRST_TRAVEL_KEY = 'first_travel'
PROBABILITIES_KEY = 'probs'

In [26]:
first_travel_dict = defaultdict(dict)
for node in tqdm(G_large.nodes()):
    first_travel_dict[node][FIRST_TRAVEL_KEY] = (A[node, :] / np.sum(A[node, :])).data
    #first_travel_dict[node][FIRST_TRAVEL_KEY].update({A[node, :].toarray()[0] / np.sum(A[node, :])})

In [28]:
p = 1
q = 1
prob_dict = defaultdict(dict)
for current_node in tqdm(G_large.nodes()):
    prob_dict[current_node][PROBABILITIES_KEY] = dict()
    dict_to_add = prob_dict[current_node][PROBABILITIES_KEY]
    for source in G_large.neighbors(current_node):
        r = {source: (A[current_node, :] + ((1/p)-1)*(A[current_node, :].multiply(A[source, :]))).data}
        dict_to_add.update(r)

In [31]:
d_graph.update(first_travel_dict)
for k,v in d_graph.items():
    v.update(prob_dict[k])

## Parallel

In [32]:
p = 1
q = 1

A = nx.adjacency_matrix(G_large)
FIRST_TRAVEL_KEY = 'first_travel'
PROBABILITIES_KEY = 'probs'

In [33]:
from joblib import Parallel, delayed

In [37]:
def get_first_travel(node):
    ftd = defaultdict(dict)
    ftd[node][FIRST_TRAVEL_KEY] = (A[node, :] / np.sum(A[node, :])).data
    return ftd

In [38]:
def get_probabilities(current_node):
    pd = defaultdict(dict)
    pd[current_node][PROBABILITIES_KEY] = dict()
    dict_to_add = pd[current_node][PROBABILITIES_KEY]
    for source in A[current_node, :].nonzero()[1]:
        r = {source: (A[current_node, :] + ((1/p)-1)*(A[current_node, :].multiply(A[source, :]))).data}
        dict_to_add.update(r)
        
    return pd

Joblib starts independent processes (or threads) without shared memory! Each process/thread receives 
a copy of the current program state. This is possible because the function does not update a shared variable. 
It only reads values from a copy of the adjacency matrix.

It is faster without shuffling the order in which the nodes are iterated over. 

In [50]:
first_travels = Parallel(n_jobs=8)(delayed(get_first_travel)(node) for node in tqdm(G_large.nodes()))

In [51]:
probs = Parallel(n_jobs=8)(delayed(get_probabilities)(node) for node in tqdm(G_large.nodes()))

In [62]:
d_graph = defaultdict(dict)
first_travel_dict = {}
prob_dict = {}

In [63]:
for x in first_travels: 
    first_travel_dict.update(x)

In [64]:
for x in probs: 
    prob_dict.update(x)

In [65]:
d_graph.update(first_travel_dict)
for k,v in d_graph.items():
    v.update(prob_dict[k])

In [66]:
d_graph

defaultdict(dict,
            {0: {'first_travel': array([0.5, 0.5]),
              'probs': {1000: array([1., 1.]), 1871: array([1., 1.])}},
             1: {'first_travel': array([0.5, 0.5]),
              'probs': {1001: array([1., 1.]), 1872: array([1., 1.])}},
             2: {'first_travel': array([0.5, 0.5]),
              'probs': {1002: array([1., 1.]), 1873: array([1., 1.])}},
             3: {'first_travel': array([0.5, 0.5]),
              'probs': {1003: array([1., 1.]), 1874: array([1., 1.])}},
             4: {'first_travel': array([0.5, 0.5]),
              'probs': {1004: array([1., 1.]), 1875: array([1., 1.])}},
             5: {'first_travel': array([0.5, 0.5]),
              'probs': {1005: array([1., 1.]), 1876: array([1., 1.])}},
             6: {'first_travel': array([0.5, 0.5]),
              'probs': {1006: array([1., 1.]), 1877: array([1., 1.])}},
             7: {'first_travel': array([0.5, 0.5]),
              'probs': {1007: array([1., 1.]), 1873: array([1.

In [61]:
n2v.d_graph

defaultdict(dict,
            {0: {'probabilities': {1000: array([0.5, 0.5]),
               1871: array([0.5, 0.5])},
              'first_travel_key': array([0.5, 0.5]),
              'neighbors': [1871, 1000]},
             1871: {'probabilities': {0: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.11111111, 0.11111111, 0.11111111]),
               225: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.11111111, 0.11111111, 0.11111111]),
               272: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.11111111, 0.11111111, 0.11111111]),
               339: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.11111111, 0.11111111, 0.11111111]),
               481: array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
                      0.11111111, 0.111111

# Small Test

In [ ]:
G = nx.karate_club_graph()
A = nx.adjacency_matrix(G)
d_graph = dict()
FIRST_TRAVEL_KEY = 'first_travel'
PROBABILITIES_KEY = 'probs'

In [ ]:
np.sum(A[0, :])

In [ ]:
# Start from the current nodes intstead of source nodes. 

first_travel_dict = {node: {FIRST_TRAVEL_KEY: A[node, :].toarray()[0] / np.sum(A[node, :])} for node in G.nodes()}
prob_dict = {current_node: {PROBABILITIES_KEY: {source: (A[curn, :] + ((1/p)-1)*(A[curn, :].multiply(A[source, :]))).toarray() for source in G.neighbors(current_node)}} for current_node in G.nodes()}
d_graph.update(first_travel_dict)


In [ ]:
for k,v in d_graph.items():
    print(v)
    v.update(prob_dict[k])
    print(v)

In [ ]:
curn = 2
source = 0
p = 2
(A[curn, :] + ((1/p)-1)*(A[curn, :].multiply(A[source, :]))).toarray()

In [ ]:
{current_node: {PROBABILITIES_KEY: {source: (A[curn, :] + ((1/p)-1)*(A[curn, :].multiply(A[source, :]))).toarray() for source in G.neighbors(current_node)}} for current_node in G.nodes()}